In [ ]:
import time
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from collections import Counter
from tqdm import tqdm

The goal is to implement a trigram model both using counts and a neural network.

How do we construct these trigrams? What is the idea if we use counts? A trigram is a sequence of three letters. We want to model the probability of seeing a particular letter given the previous two. How do we do that for the beginning of a word? Does a word begin with two '.' elements?

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [ ]:
default_dtype = torch.float32
torch.set_default_dtype(default_dtype)

In [ ]:
with open('names.txt', 'r') as file:
    words = file.read().splitlines()

In [ ]:
def trigrams(words):
    for w in words:
        chs = ['.', '.'] + list(w) + ['.']
        for c1, c2, c3 in zip(chs, chs[1:], chs[2:]):
            yield c1, c2, c3

In [ ]:
chars = sorted(list(set(''.join(words))))

A trigram count model would map two chars to a single char that follows. What dimensions should a count lookup table have? Well, what are all the possible two char sequences that we might have? Certainly can be two dots or it can start with a dot. Can't start with a letter and end with a dot, because that should have terminated evaluation earlier. So the number should be $1\cdot1 + 1\cdot26 + 26\cdot26 = 27\cdot26 + 1 = 27\cdot27-26$.

In [ ]:
DOT = '.'

In [ ]:
btoi = {}
i = 0

btoi[(DOT, DOT)] = i
i += 1

for c in chars:
    btoi[(DOT, c)] = i
    i += 1
    
for c1 in chars:
    for c2 in chars:
        btoi[(c1, c2)] = i
        i += 1

In [ ]:
itob = {i: b for b, i in btoi.items()}

In [ ]:
ctoi = {}
i = 0

ctoi[DOT] = i
i += 1

for c in chars:
    ctoi[c] = i
    i += 1
    
itoc = {i: c}

In [ ]:
itoc = {i: c for c, i in ctoi.items()}

In [ ]:
m = len(itob)
n = len(itoc)

# Count-based model

In [ ]:
N = torch.zeros((m, n), dtype=torch.int32, device='cpu')

In [ ]:
for c1, c2, c3 in trigrams(words):
    i1 = btoi[(c1, c2)]
    i2 = ctoi[c3]
    N[i1, i2] += 1

In [ ]:
P = N.float()
P /= P.sum(axis=1, keepdim=True)

In [ ]:
g = torch.Generator(device='cpu').manual_seed(42)

def makeone():
    i = 0
    s = ''
    while True:
        p = P[i]
        ci = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        # now we have a new character index, we need to update our lookup bigram
        # the new lookup bigram will contain the second character in the first position
        # and the new character in the second position
        # what is the current bigram? it is given by i
        if ci == 0:
            break
        i = btoi[(itob[i][1], itoc[ci])]
        s += itoc[ci]
    return s

Now want to evaluate the model. The idea is to calculate the likelihood of the dataset given the model parameters.

In [ ]:
ll = 0
k = 0
for word in words:
    chs = ['.', '.'] + list(word) + ['.']
    for c1, c2, c3 in zip(chs, chs[1:], chs[2:]):
        i1 = btoi[(c1, c2)]
        i2 = ctoi[c3]
        ll += torch.log(P[i1, i2])
        k += 1
print(f"mean negative likelihood is {-ll/k}")

# Gradient descent optimization

Create a dataset. We do this by turning character indices into one hot vector. The first character in a bigram is an $x$, the second character is a $y$.

In [ ]:
xs = []
ys = []
for word in words:
    chs = ['.', '.'] + list(word) + ['.']
    for c1, c2, c3 in zip(chs, chs[1:], chs[2:]):
        i1 = btoi[(c1, c2)]
        i2 = ctoi[c3]
        xs.append(i1)
        ys.append(i2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [ ]:
xenc = F.one_hot(xs, m).to(default_dtype)

In [ ]:
g = torch.Generator(device='cuda').manual_seed(2147483647)
W = torch.randn(m, n, generator=g, requires_grad=True)

In [ ]:
t = time.time()
print(f"{'epoch':>6} {'loss':>10} {'time,s':>7}")
for i in range(100000):
    logits = xenc @ W # log-counts
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(len(xs)), ys].log().mean()# + 20*(W**2).mean()
    if (i+1)%1000 == 1:
        tt = time.time()
        print(f"{i+1:6} {loss.data.item():10.5f} {tt-t:>7.2f}")
        t = tt

    W.grad = None # zero out the gradients
    loss.backward()
    W.data += -50*W.grad

In [ ]:
i = 0
s = ''
while True:
    xenc = F.one_hot(torch.tensor([i]), m).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    ci = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
    if ci == 0:
        break
    i = btoi[(itob[i][1], itoc[ci])]
    s += itoc[ci]
print(s)